In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('telco.db')
cursor = conn.cursor()

# 1. สร้างตารางแพ็กเกจ (Packages)
cursor.execute('''CREATE TABLE IF NOT EXISTS packages (
    package_id INTEGER PRIMARY KEY,
    package_name TEXT,
    price INTEGER)''')

# 2. สร้างตารางลูกค้า (Subscribers)
cursor.execute('''CREATE TABLE IF NOT EXISTS subscribers (
    sub_id INTEGER PRIMARY KEY,
    name TEXT,
    package_id INTEGER)''')

# 3. สร้างตารางการใช้งานเน็ต (Data_Usage) - บันทึกการใช้รายวัน
cursor.execute('''CREATE TABLE IF NOT EXISTS data_usage (
    usage_id INTEGER PRIMARY KEY,
    sub_id INTEGER,
    data_gb REAL)''')

# 4. ใส่ข้อมูลจำลอง
# แพ็กเกจ: 1=Starter(299), 2=Basic(499), 3=MaxSpeed(899), 4=Unlimited(1099)
cursor.executemany('INSERT INTO packages VALUES (?,?,?)', [
    (1, 'Starter', 299), (2, 'Basic', 499), (3, 'MaxSpeed', 899), (4, 'Unlimited', 1099)])

# ลูกค้า
cursor.executemany('INSERT INTO subscribers VALUES (?,?,?)', [
    (101, 'Keng', 1), (102, 'Bum', 1), (103, 'Ann', 2), 
    (104, 'Boy', 3), (105, 'Cat', 1), (106, 'Dan', 4)])

# การใช้งานเน็ต (ใครใช้เยอะๆ บ้าง?)
# Keng (โปร 299) ใช้ไป 20+35 = 55GB (เป้าหมาย!)
# Bum (โปร 299) ใช้ไป 5+5 = 10GB
# Ann (โปร 499) ใช้ไป 40+20 = 60GB (เป้าหมาย! เพราะราคา < 500)
# Boy (โปร 899) ใช้ไป 100GB (ไม่เอา เพราะจ่ายแพงอยู่แล้ว)
usage_data = [
    (1, 101, 20.5), (2, 101, 35.0), # Keng
    (3, 102, 5.0), (4, 102, 5.0),   # Bum
    (5, 103, 40.0), (6, 103, 20.0), # Ann
    (7, 104, 100.0),                # Boy
    (8, 105, 60.0),                 # Cat (โปร 299, ใช้ 60GB -> เป้าหมาย!)
    (9, 106, 200.0)                 # Dan
]
cursor.executemany('INSERT INTO data_usage VALUES (?,?,?)', usage_data)
conn.commit()
print("✅ ระบบ Telco Database พร้อมใช้งาน!")

✅ ระบบ Telco Database พร้อมใช้งาน!


In [2]:
print("--- 1. ตารางแพ็กเกจ (Packages) ---")
df_packages = pd.read_sql("SELECT * FROM packages", conn)
display(df_packages)

--- 1. ตารางแพ็กเกจ (Packages) ---


,package_id,package_name,price
0,1,Starter,299
1,2,Basic,499
2,3,MaxSpeed,899
3,4,Unlimited,1099


In [3]:
print("\n--- 2. ตารางลูกค้า (Subscribers) ---")
df_subscribers = pd.read_sql("SELECT * FROM subscribers", conn)
display(df_subscribers)


--- 2. ตารางลูกค้า (Subscribers) ---


,sub_id,name,package_id
0,101,Keng,1
1,102,Bum,1
2,103,Ann,2
3,104,Boy,3
4,105,Cat,1
5,106,Dan,4


In [4]:
print("\n--- 3. ตารางการใช้งาน (Data Usage) ---")
df_usage = pd.read_sql("SELECT * FROM data_usage", conn)
display(df_usage)


--- 3. ตารางการใช้งาน (Data Usage) ---


,usage_id,sub_id,data_gb
0,1,101,20.5
1,2,101,35.0
2,3,102,5.0
3,4,102,5.0
4,5,103,40.0
5,6,103,20.0
6,7,104,100.0
7,8,105,60.0
8,9,106,200.0


In [5]:
#ข้อมูลแบบเชื่อมตารางดูไหม?

In [6]:
print("\n--- 📊 สรุปยอดการใช้งานของแต่ละคน (Join Tables) ---")

sql = """
SELECT 
    subscribers.name, 
    packages.package_name, 
    packages.price,
    SUM(data_usage.data_gb) as total_data_gb
FROM subscribers
INNER JOIN packages ON subscribers.package_id = packages.package_id
LEFT JOIN data_usage ON subscribers.sub_id = data_usage.sub_id
GROUP BY subscribers.name
ORDER BY total_data_gb DESC
"""

df_result = pd.read_sql(sql, conn)
display(df_result)


--- 📊 สรุปยอดการใช้งานของแต่ละคน (Join Tables) ---


,name,package_name,price,total_data_gb
0,Dan,Unlimited,1099,200.0
1,Boy,MaxSpeed,899,100.0
2,Cat,Starter,299,60.0
3,Ann,Basic,499,60.0
4,Keng,Starter,299,55.5
5,Bum,Starter,299,10.0


In [7]:
# โจทย์: ตามหาลูกค้าเป้าหมาย (ใช้เน็ตเยอะ > 50GB แต่จ่ายน้อย < 500 บาท)

sql = """
SELECT 
    subscribers.name, 
    packages.package_name, 
    packages.price, 
    SUM(data_usage.data_gb) AS Total_Data_GB
FROM subscribers
INNER JOIN packages ON subscribers.package_id = packages.package_id
INNER JOIN data_usage ON subscribers.sub_id = data_usage.sub_id
WHERE packages.price < 500
GROUP BY subscribers.name
HAVING Total_Data_GB > 50
ORDER BY Total_Data_GB DESC
"""

# รันผลลัพธ์
df_target = pd.read_sql(sql, conn)
display(df_target)

,name,package_name,price,Total_Data_GB
0,Cat,Starter,299,60.0
1,Ann,Basic,499,60.0
2,Keng,Starter,299,55.5
